<a href="https://colab.research.google.com/github/mysticizhere/ML/blob/main/Tensorflow_Classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

Defining some constants to help later on: 

In [3]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

Feeding data Set :

In [4]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


Now we can pop the species column off and use that as our label :

In [7]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [8]:
train.shape

(120, 4)

In [20]:
print(train.keys())

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')


**Feature Columns**

In [24]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    print(key)
print(my_feature_columns)

SepalLength
SepalWidth
PetalLength
PetalWidth
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


Or you can use the following way as well 

In [18]:
feature_col = []
for feature_name in CSV_COLUMN_NAMES:
  feature_col.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_col)  

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Species', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


**Input Function**

In [25]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    dataset = dataset.batch(batch_size)
    return dataset

Building the model using DNN classifier

In [26]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

**Training**

In [31]:
train_input_fn = lambda: input_fn(train, train_y, training=True)

In [32]:
classifier.train(
    train_input_fn,
    steps=5000)
# We include a lambda to avoid creating an inner function previously

**Testing**

In [34]:
test_input_fn = lambda: input_fn(test, test_y, training=False)

In [35]:
result = classifier.evaluate(test_input_fn)

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**result))


Test set accuracy: 0.967



Prediction for Individual Data :

In [37]:
def new_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}
#taking the input of features and storing them in predict dict.
print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]
print(predict)  #predict contains all the inputs fed into it.
#Making predict input fn for predict dict.  
predict_input_fn = lambda: new_fn(predict)
predictions = classifier.predict(predict_input_fn)#predictions contains all the results that tells us about particular predict dict. (i.e result)
#but we only want to see the species and probability so we run a for loop as follows :
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0] # it will be either 0,1 or 2 corresponding to each type of flower in species list.
    print(pred_dict)
    probability = pred_dict['probabilities'][class_id] # this will give the probability of the class id predicted by our model

    print('Prediction is "{}" ({:.1f}%)'.format( #formatting code, useless 
        SPECIES[class_id], 100 * probability)) # we access name of the species through the species list then its probability

Please type numeric values as prompted.
SepalLength: 2.1
SepalWidth: 1.4
PetalLength: 5.2
PetalWidth: 2.3
{'SepalLength': [2.1], 'SepalWidth': [1.4], 'PetalLength': [5.2], 'PetalWidth': [2.3]}
{'logits': array([-6.5352597 , -0.35155466,  4.481881  ], dtype=float32), 'probabilities': array([1.6287946e-05, 7.8961505e-03, 9.9208754e-01], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (99.2%)
